# Create Dim Date Dimensions

In [0]:
import requests
import json
from datetime import datetime, timedelta

In [0]:
# Define constants
BASE_URL = "https://openholidaysapi.org/PublicHolidays"
COUNTRY_CODE = "CH"
LANGUAGE_CODE = "DE"
SUBDIVISION_CODE = "CH-ZH"
# EXCLUDED_NAMES = ["Eidgenössischer Dank-, Buss- und Bettag", "Berchtoldstag"]
EXCLUDED_NAMES = []
# MANUAL_DATES = [('2024-01-02', 'Kompensation', 'Example Kompensation')]
MANUAL_DATES = []

In [0]:
# Function to fetch holidays from the API
def fetch_holidays(start_date, end_date):
    params = {
        "countryIsoCode": COUNTRY_CODE,
        "languageIsoCode": LANGUAGE_CODE,
        "validFrom": start_date,
        "validTo": end_date
    }
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    return response.json()

# Function to filter holidays
def filter_holidays(holidays):
    return [
        {
            "date": holiday["startDate"],
            "type": "PublicHoliday",
            "name": next((n["text"] for n in holiday["name"] if n["language"] == LANGUAGE_CODE), None)
        }
        for holiday in holidays
        if holiday["nationwide"] or (
            not holiday["nationwide"] and
            any(sub["code"] == SUBDIVISION_CODE for sub in holiday.get("subdivisions", [])) and
            next((n["text"] for n in holiday["name"] if n["language"] == LANGUAGE_CODE), None) not in EXCLUDED_NAMES
        )
    ]

In [0]:
def main(start_date, end_date, output_file):
    all_holidays = []

    # Convert start and end dates to datetime objects
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")

    # Loop through the date range in 2-year increments
    while start_dt < end_dt:
        current_end_dt = min(start_dt + timedelta(days=2*365), end_dt)
        current_start_date = start_dt.strftime("%Y-%m-%d")
        current_end_date = current_end_dt.strftime("%Y-%m-%d")

        # Fetch holidays for the current 2-year period
        holidays = fetch_holidays(current_start_date, current_end_date)
        all_holidays.extend(filter_holidays(holidays))

        # Move to the next 2-year period
        start_dt = current_end_dt + timedelta(days=1)

    # Append manual dates
    # all_holidays.extend({"date": date, "type": holiday_type, "name": name} for date, holiday_type, name in MANUAL_DATES)

    # Sort holidays by date
    all_holidays.sort(key=lambda x: x["date"])

    # Save to JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_holidays, f, ensure_ascii=False, indent=4)
    print(f"Holidays saved to {output_file}")

In [0]:
if __name__ == "__main__":
    start_date = "2004-01-01"
    end_date = "2030-12-31"
    output_file = "dates.json"
    main(start_date, end_date, output_file)